In [1]:
import os
import numpy
import random
import json
from distutils.dir_util import copy_tree
import shutil
import time

import sys, os
sys.path.append(os.path.dirname('../.'))
from scripts.utils import get_path

In [2]:
path_to_data = '/home/msst/Documents/medtech/data/TestModels_IXI/data_for_tests'

In [6]:
def create_test(path_to_data, n_train, n_val, n_test):
    sample_names = set()
    for name in os.listdir(path_to_data):
        if not os.path.isfile(f"{path_to_data}/{name}"):
            sample_names = sample_names.union([name[:6],])

    to_train = random.sample(list(sample_names), k=n_train)
    sample_names = sample_names.difference(to_train)
    to_val = random.sample(list(sample_names), k=n_val)    
    sample_names = sample_names.difference(to_val)
    to_test = random.sample(list(sample_names), k=n_test) 
    
    test_dict = {
        'train_patches_per_volume': None,
        'train': [],
        'val': [],
        'test': []
    }

    if n_train==1: test_dict['train_patches_per_volume'] = 256
    if n_train==2: test_dict['train_patches_per_volume'] = 196
    if n_train>2: test_dict['train_patches_per_volume'] = 128
    if n_train>4: test_dict['train_patches_per_volume'] = 64

    
    for name in to_train:
        test_dict['train'].append(f'{path_to_data}/{name}_0')
        test_dict['train'].append(f'{path_to_data}/{name}_1')
    for name in to_val:
        test_dict['val'].append(f'{path_to_data}/{name}_0')
    for name in to_test:
        test_dict['test'].append(f'{path_to_data}/{name}_0')

    return(test_dict)

In [7]:
tests = {}
tests.update({'test1' : create_test(path_to_data, n_train=1, n_val=2, n_test=3)})
tests.update({'test2' : create_test(path_to_data, n_train=1, n_val=2, n_test=3)})
tests.update({'test3' : create_test(path_to_data, n_train=1, n_val=2, n_test=3)})
tests.update({'test4' : create_test(path_to_data, n_train=2, n_val=2, n_test=3)})
tests.update({'test5' : create_test(path_to_data, n_train=2, n_val=2, n_test=3)})
tests.update({'test6' : create_test(path_to_data, n_train=2, n_val=2, n_test=3)})
tests.update({'test7' : create_test(path_to_data, n_train=4, n_val=2, n_test=3)})
tests.update({'test8' : create_test(path_to_data, n_train=4, n_val=2, n_test=3)})
tests.update({'test9' : create_test(path_to_data, n_train=4, n_val=2, n_test=3)})

In [8]:
fp2 = open(f"./testing_info.json", 'w')
json.dump(tests, sort_keys=True, indent=4, fp=fp2)
fp2.close()

In [72]:
path_to_processed_data = '/home/msst/Documents/medtech/data/HessData_IXI/DATA/processed'
path_to_datasets = '/home/msst/Documents/medtech/data/HessData_IXI_test'

if os.path.exists(path_to_datasets):
    shutil.rmtree(path_to_datasets)
    
if not os.path.exists(path_to_datasets):
    os.mkdir(path_to_datasets)

for test_key in testing_info_dict:
    path_to_set = path_to_datasets+'/'+test_key
    
    if not os.path.exists(path_to_set):
        os.mkdir(path_to_set)
        os.mkdir(path_to_set+"/train")
        os.mkdir(path_to_set+"/test")
        
    
    for name in testing_info_dict[test_key]['train']:
        for tree in get_path(path_to_processed_data, name):
            p = f"{path_to_set}/train/{os.path.basename(tree)}"
            if not os.path.exists(p):
                os.mkdir(p)
            copy_tree(tree, p)

    for name in testing_info_dict[test_key]['test']:
        for tree in get_path(path_to_processed_data, name):
            p = f"{path_to_set}/test/{os.path.basename(tree)}"
            if p[-1]=='0':
                if not os.path.exists(p):
                    os.mkdir(p)
                copy_tree(tree, p)
    

In [3]:
def parse_avg_metrics(avg_metrics_path):
    avg_metrics_dict = json.load(open(avg_metrics_path))
    for model in avg_metrics_dict:
        avg_metrics = {}
        avg_metrics_mask = {}
        print("\nMODEL:", model)
        tests = avg_metrics_dict[model]
        for test in tests:
            data = tests[test]
            for metric in data['avg_metrics']:
                if not avg_metrics.get(metric):
                    avg_metrics.update({metric: [data['avg_metrics'][metric],]})
                else:
                    avg_metrics[metric].append(data['avg_metrics'][metric])
            for metric in data['avg_metrics_mask']:
                if not avg_metrics_mask.get(metric):
                    avg_metrics_mask.update({metric: [data['avg_metrics_mask'][metric],]})
                else:
                    avg_metrics_mask[metric].append(data['avg_metrics_mask'][metric])
        
        
        print("\tavg_metrics:")
        for m in avg_metrics:
            avg_metrics[m] = sum([float(x) for x in avg_metrics[m]])/len(avg_metrics[m])
            print('\t\t', m, ":", avg_metrics[m])
        
        print("\tavg_metrics_mask:")
        for m in avg_metrics_mask:
            avg_metrics_mask[m] = sum([float(x) for x in avg_metrics_mask[m]])/len(avg_metrics_mask[m])
            print('\t\t', m, ":", avg_metrics_mask[m])
            
        #return({"avg_metrics" : avg_metrics, 
        #        "avg_metrics_mask" : avg_metrics_mask})

In [5]:
avg_metrics_path = '/home/msst/Documents/medtech/data/TestModels_IXI/out/avg_metrics.json'
_ = parse_avg_metrics(avg_metrics_path)


MODEL: HessNet
	avg_metrics:
		 AVGDIST : 1.9389134074074075
		 DICE : 0.7790607407407408
		 SNSVTY : 0.8487398888888891
	avg_metrics_mask:
		 AVGDIST : 0.47405266666666673
		 DICE : 0.8531573333333333
		 SNSVTY : 0.869843148148148

MODEL: JustConv
	avg_metrics:
		 AVGDIST : 3.077104481481481
		 DICE : 0.6812497037037037
		 SNSVTY : 0.721489111111111
	avg_metrics_mask:
		 AVGDIST : 0.6172261851851851
		 DICE : 0.7933803333333332
		 SNSVTY : 0.9141489999999999

MODEL: TwoConv
	avg_metrics:
		 AVGDIST : 1.2807137407407407
		 DICE : 0.7663785925925926
		 SNSVTY : 0.7278086666666668
	avg_metrics_mask:
		 AVGDIST : 0.847164074074074
		 DICE : 0.7876780740740741
		 SNSVTY : 0.8879016666666666

MODEL: UNETR
	avg_metrics:
		 AVGDIST : 8.976208666666668
		 DICE : 0.7305721111111111
		 SNSVTY : 0.8526773703703704
	avg_metrics_mask:
		 AVGDIST : 0.3557209259259259
		 DICE : 0.8747555185185184
		 SNSVTY : 0.902607074074074

MODEL: Unet2d_16ch
	avg_metrics:
		 AVGDIST : 0.9140291481481483
		 DICE 